#### In this report, our team aims to build a *Gradient Boosting Model* to forecast traffic conditions, specifically congestions, at a specific location, time, and for the Jakarta dataset, what kind of vehicle they are driving. 
#### With this model, we wish to provide Grab with the ability to provide its employees with future forecasts of congested roads to avoid and less congested roads to take. Additionally, it could also have the ability to inform its users of potential delays in arrivals and by how much, all depending on that specific time of the day, day of the week, location the driver is right now or might be passing through later. 
#### Eg.) The current time is 6pm, and the Grab driver is at Singapore's Central Business District. We believe our model could predict that there will be congestions, which is also the logical thought, and with this prediction, our model can find a region with lower congestino that the driver can take, and also let the user know that the delivery/arrival might be later than expected due congestion. 

## Loading in all the different datasets and all needed libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import sklearn
import datetime

def to_category(df, *args):
    for col_name in args:
        df[col_name] = df[col_name].astype("category")
    
def to_float32(df, *args):
    for col_name in args:
        df[col_name] = df[col_name].astype("float32")
        
def to_uint16(df, *args):
    for col_name in args:
        df[col_name] = df[col_name].astype("uint16")

def to_int32(df, *args):
    for col_name in args:
        df[col_name] = df[col_name].astype("int32")

def format_datetime(df, col_name):
    # get datetime obj for all timestamps
    dt = df[col_name].apply(datetime.datetime.fromtimestamp)
    
    df["time"] = dt.apply(lambda x: x.time())
    df["day_of_week"] = dt.apply(lambda x: x.weekday())
    df["month"] = dt.apply(lambda x: x.month)
    df["year"] = dt.apply(lambda x: x.year)

sg_0 = pd.read_parquet('singapore/part-00000-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
sg_1 = pd.read_parquet('singapore/part-00001-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
sg_2 = pd.read_parquet('singapore/part-00002-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
sg_3 = pd.read_parquet('singapore/part-00003-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
sg_4 = pd.read_parquet('singapore/part-00004-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
sg_5 = pd.read_parquet('singapore/part-00005-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
sg_6 = pd.read_parquet('singapore/part-00006-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
sg_7 = pd.read_parquet('singapore/part-00007-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
sg_8 = pd.read_parquet('singapore/part-00008-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
sg_9 = pd.read_parquet('singapore/part-00009-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')

jk_0 = pd.read_parquet('jakarta/part-00000-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
jk_1 = pd.read_parquet('jakarta/part-00001-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
jk_2 = pd.read_parquet('jakarta/part-00002-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
jk_3 = pd.read_parquet('jakarta/part-00003-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
jk_4 = pd.read_parquet('jakarta/part-00004-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
jk_5 = pd.read_parquet('jakarta/part-00005-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
jk_6 = pd.read_parquet('jakarta/part-00006-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
jk_7 = pd.read_parquet('jakarta/part-00007-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
jk_8 = pd.read_parquet('jakarta/part-00008-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')
jk_9 = pd.read_parquet('jakarta/part-00009-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet', engine='fastparquet')

sg = [sg_0, sg_1, sg_2, sg_3, sg_4, sg_5, sg_6, sg_7, sg_8, sg_9] #only car for driving mode
jk = [jk_0, jk_1, jk_2, jk_3, jk_4, jk_5, jk_6, jk_7, jk_8, jk_9] #has car and motorcycle for driving mode

for data in sg:
    format_datetime(data, "pingtimestamp")
    to_category(data, ["trj_id", "driving_mode", "osname"])
    to_float32(data, ["rawlat", "rawlng", "speed", "accuracy"])
    to_uint16(data, ["bearing", "day_of_week", "month", "year"])
    to_int32(data, "pingtimestamp")
    data.time = pd.to_datetime(data['time'], format='%H:%M:%S')
    data.time = data.time.dt.hour
    
for data in jk:
    format_datetime(data, "pingtimestamp")
    to_category(data, ["trj_id", "driving_mode", "osname"])
    to_float32(data, ["rawlat", "rawlng", "speed", "accuracy"])
    to_uint16(data, ["bearing", "day_of_week", "month", "year"])
    to_int32(data, "pingtimestamp")
    data.time = pd.to_datetime(data['time'], format='%H:%M:%S')
    data.time = data.time.dt.hour

### 1. Data Preparation (training - 60%, validation - 20%, test - 20%)

In [2]:
#For Singapore model
training_set_sg = [sg_0, sg_1, sg_2, sg_3, sg_4, sg_5]
training_merge_sg = pd.concat(training_set_sg)

validation_set_sg = [sg_6, sg_7]
validation_merge_sg = pd.concat(validation_set_sg)

test_set_sg = [sg_8, sg_9]
test_merge_sg = pd.concat(test_set_sg)

#For Jakarta model
training_set_jk = [jk_0, jk_1, jk_2, jk_3, jk_4, jk_5]
training_merge_jk = pd.concat(training_set_jk)

validation_set_jk = [jk_6, jk_7]
validation_merge_jk = pd.concat(validation_set_jk)

test_set_jk = [jk_8, jk_9]
test_merge_jk = pd.concat(test_set_jk)

### 2. Feature Engineering + Data Cleaning

> #### Target Variable: 'speed'
> #### Feature Variable for Singapore: 'rawlat', 'rawlng', 'time', 'day_of_week'
> #### Feature Variable for Jakarta: 'driving_mode', 'rawlat', 'rawlng', 'time', 'day_of_week'

> #### However, I would like to mention there are '-1' for speed which is considered as Invalid. We have decided to drop '-1' values in the dataset as we believe removing them would be the most accurate and reliable action to take. In fact, we decided to remove all negative speed value for a clearer and more accurate picture. 

In [3]:
datasets = [training_merge_sg, validation_merge_sg, test_merge_sg, training_merge_jk, validation_merge_jk, test_merge_jk]

In [4]:
training_merge_sg = training_merge_sg[training_merge_sg['speed'] >= 0]
validation_merge_sg = validation_merge_sg[validation_merge_sg['speed'] >= 0]
test_merge_sg = test_merge_sg[test_merge_sg['speed'] >= 0]
training_merge_jk = training_merge_jk[training_merge_jk['speed'] >= 0]
validation_merge_jk = validation_merge_jk[validation_merge_jk['speed'] >= 0]
test_merge_jk = test_merge_jk[test_merge_jk['speed'] >= 0]

In [5]:
singapore_features = ['rawlat', 'rawlng', 'time', 'day_of_week']
singapore_target = ['speed']

> ##### one hot encoding for 'driving_mode' for jakarta datasets

In [6]:
one_hot = pd.get_dummies(training_merge_jk['driving_mode'])
training_merge_jk = training_merge_jk.drop('driving_mode', axis = 1)
training_merge_jk = pd.concat([training_merge_jk, one_hot], axis=1)

In [7]:
one_hot = pd.get_dummies(validation_merge_jk['driving_mode'])
validation_merge_jk = validation_merge_jk.drop('driving_mode', axis = 1)
validation_merge_jk = pd.concat([validation_merge_jk, one_hot], axis=1)

In [8]:
one_hot = pd.get_dummies(test_merge_jk['driving_mode'])
test_merge_jk = test_merge_jk.drop('driving_mode', axis = 1)
test_merge_jk = pd.concat([test_merge_jk, one_hot], axis=1)

In [9]:
jakarta_features = ['rawlat', 'rawlng', 'time', 'day_of_week', 'car', 'motorcycle']
jakarta_target = ['speed']

> #### Seems like we got all our data prepared for both the Singapore and Jakarta Model 

### 3.1 Training the model for Singapore

> ##### This would be a supervised learning task and amongst the different models that can be used, we have decided to proceed with Gradient Boosting (ensemble method).
> ##### The reasons we have chose Gradient Boosting is firstly because we are dealing with latitude and longitude data so we wanted to pick a model which was insensitive to its features being scaled. Secondly, Gradient Boosting is a very robust model that aims to improve on its previous attempt and provides unmatched level of accuracy. Additionally, it provides numerous hyperparameters that we can possibly tune in order to sharpen our model to give us the best predictions

In [33]:
#Singapore
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
model = XGBRegressor(random_state=42, n_jobs=-1)

#Fit the algorithm on the data
model.fit(training_merge_sg[singapore_features], training_merge_sg[singapore_target])

#Predict training set:
preds = model.predict(training_merge_sg[singapore_features])

#Evaluation
error = mean_squared_error(preds, training_merge_sg[singapore_target], squared=False)

print("Training mean error for sg is {}".format(error))

C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Training mean error for sg is 5.36489725112915


In [34]:
#Validation

preds = model.predict(validation_merge_sg[singapore_features])

error_val = mean_squared_error(validation_merge_sg[singapore_target], preds,squared = False)

print("Validation mean error for sg is {}".format(error_val))

C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Validation mean error for sg is 5.3665056228637695


> #### We have created a simple naive predicting model that predicts all values as the mean value of the 'speed' column. This model will be the benchmark for our own model to be evaluated on. 

In [35]:
#Create a naive predicting model 
naive_pred = np.full((len(validation_merge_sg[singapore_target]), 1), training_merge_sg.speed.mean())

In [36]:
naive_error = mean_squared_error(validation_merge_sg[singapore_target], naive_pred, squared = False)
print("Naive mean error for sg is {}".format(naive_error))

Naive mean error for sg is 7.166916370391846


> #### Since using the tens of millions of rows would be extremely time consuming for hyperparameter tuning, we decided to select a batch of data from our engineered datasets to obtain the optimal hyperparameters before we apply it on the entire dataset

In [37]:
import xgboost as xgb
dtrain = xgb.DMatrix(training_merge_sg.copy()[:10000][singapore_features], label=training_merge_sg.copy()[:10000][singapore_target])
dtest = xgb.DMatrix(validation_merge_sg.copy()[:10000][singapore_features], label=validation_merge_sg.copy()[:10000][singapore_target])

C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [38]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
    'eval_metric': 'mae'
}
num_boost_round = 999

cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)
cv_results

[10:12:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:12:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:12:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:12:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:12:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
0,11.203004,0.014374,11.223943,0.072649
1,8.723354,0.003951,8.780243,0.047574
2,7.167634,0.011117,7.251382,0.047283
3,6.152725,0.018019,6.279767,0.038728
4,5.495252,0.032019,5.658433,0.053401
...,...,...,...,...
385,1.011491,0.011481,2.975566,0.038943
386,1.008975,0.011197,2.974794,0.038223
387,1.007035,0.010375,2.973893,0.038514
388,1.005157,0.009342,2.973393,0.038780


In [39]:
cv_results['test-mae-mean'].min()

2.9733052

In [40]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(7,12)
    for min_child_weight in range(5,8)
]
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=7, min_child_weight=5
[10:12:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:12:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:12:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:12:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:12:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 2.8999382000000002 for 356 rounds
CV with max_depth=7, min_child_weigh

In [41]:
params['max_depth'] = 11
params['min_child_weight'] = 6

In [42]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[10:16:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:16:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:16:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:16:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:16:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 2.6366672 for 380 rounds
CV with subsample=1.0, colsample=0.9
[10:16:39] 

In [43]:
params['subsample'] = .9
params['colsample_bytree'] = 1.

In [44]:
%time
# This can take some time…
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    %time cv_results = xgb.cv(params,dtrain,num_boost_round=num_boost_round,seed=42,nfold=5,metrics=['mae'],early_stopping_rounds=10)
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

Wall time: 0 ns
CV with eta=0.3
[10:20:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:20:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:20:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:20:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[10:20:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
Wall time: 13.1 s
	MAE 2.6152404000000002 for 266 rounds

CV with eta=0.2
[10:20:34

In [45]:
params['eta'] = .3 #in this case, we would be assigning eta as 0.3 as lowering it further would only slow down our model training process

> #### Based on our small batch of data (which we acknowledge that might not be representative of the whole dataset), we have concluded on our best hyperparameters as shown below

In [47]:
params = {
    'max_depth':11,
    'min_child_weight': 6,
    'eta':.3,
    'subsample': 0.9,
    'colsample_bytree': 1,
}

> #### However, as acknowledged, the limitation(mentioned right above) seems to be more profound than usual so we had to perform some manual, logical tuning to obtain the optimal hyperparameters which are shown as below.

In [54]:
model = XGBRegressor(random_state=42, n_jobs=-1, max_depth = 11, min_child_weight = 6, subsample = 0.9, colsample_bytree = 1, eta = 0.3)

#Fit the algorithm on the data
model.fit(training_merge_sg[:500000][singapore_features], training_merge_sg[:500000][singapore_target])

#Predict training set:
preds = model.predict(training_merge_sg[:500000][singapore_features])

#Evaluation
error = mean_squared_error(preds, training_merge_sg[:500000][singapore_target], squared=False)

print("Training mean error after tuning for sg is {}".format(error))

C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Training mean error after tuning for sg is 3.7312185764312744


In [55]:
#Validation

preds = model.predict(validation_merge_sg[:500000][singapore_features])

error_val = mean_squared_error(validation_merge_sg[:500000][singapore_target], preds,squared = False)

print("Validation mean error after tuning for sg is {}".format(error_val))

C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Validation mean error for sg is 4.60477352142334


> #### Now, we will be applying this tuned model to the entire Singapore test dataset to observe whether there has been any improvement

In [56]:
tuned_model_sg = XGBRegressor(random_state=42, n_jobs=-1, max_depth = 11, min_child_weight = 6, subsample = 0.9, colsample_bytree = 1, eta = 0.3)

#Fit the algorithm on the data
tuned_model_sg.fit(test_merge_sg[singapore_features], test_merge_sg[singapore_target])

#Predict test set:
preds = tuned_model_sg.predict(test_merge_sg[singapore_features])

#Evaluation
final_test_error_sg = mean_squared_error(preds, test_merge_sg[singapore_target], squared=False)

print("Test mean error for sg is {}".format(final_test_error_sg))

C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Test mean error for sg is 4.832284450531006


### 3.2 Training the model for Jakarta

In [57]:
model = XGBRegressor(random_state=42, n_jobs=-1)

#Fit the algorithm on the data
model.fit(training_merge_jk[jakarta_features], training_merge_jk[jakarta_target])

#Predict training set:
preds = model.predict(training_merge_jk[jakarta_features])

#Evaluation
error = mean_squared_error(preds, training_merge_jk[jakarta_target], squared=False)

print("Training mean error for jk is {}".format(error))

C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Training mean error for jk is 4.379884719848633


In [58]:
#Validation

preds = model.predict(validation_merge_jk[jakarta_features])

error_val = mean_squared_error(validation_merge_jk[jakarta_target], preds,squared = False)

print("Validation mean error for jk is {}".format(error_val))

C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Validation mean error for jk is 4.379709243774414


In [59]:
#Create a naive predicting model 
naive_pred = np.full((len(test_merge_jk[jakarta_target]), 1), test_merge_jk.speed.mean())

naive_error_jk = mean_squared_error(test_merge_jk[jakarta_target], naive_pred, squared = False)
print("Naive mean error for jk is {}".format(naive_error_jk))

Naive mean error for jk is 5.177284240722656


> #### Seems like the error is generally lower for Jakarta datasets, including the naive model's error. 
> #### Similar to our Singapore model, we would first use the same hyperparameters

In [60]:
model = XGBRegressor(random_state=42, n_jobs=-1, max_depth = 11, min_child_weight = 6, subsample = 0.9, colsample_bytree = 1, eta = 0.3)

#Fit the algorithm on the data
model.fit(training_merge_jk[:500000][jakarta_features], training_merge_jk[:500000][jakarta_target])

#Predict training set:
preds = model.predict(training_merge_jk[:500000][jakarta_features])

#Evaluation
error = mean_squared_error(preds, training_merge_jk[:500000][jakarta_target], squared=False)

print("Training mean error after basic tuning for jk is {}".format(error))

C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Training mean error after basic tuning for jk is 2.6705119609832764


In [61]:
#Validation

preds = model.predict(validation_merge_jk[:500000][jakarta_features])

error_val = mean_squared_error(validation_merge_jk[:500000][jakarta_target], preds,squared = False)

print("Validation mean error after basic tuning for jk is {}".format(error_val))

C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Validation mean error after basic tuning for jk is 3.8820765018463135


> #### After some hyperparameter tuning

In [62]:
dtrain = xgb.DMatrix(training_merge_jk[:10000][jakarta_features], label=training_merge_jk[:10000][jakarta_target])
dtest = xgb.DMatrix(validation_merge_jk[:10000][jakarta_features], label=validation_merge_jk[:10000][jakarta_target])

C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [63]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
    'eval_metric': 'mae'
}
num_boost_round = 999

cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)
cv_results

[11:08:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:08:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:08:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:08:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:08:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
0,5.438444,0.014177,5.450764,0.070605
1,4.437805,0.004473,4.470284,0.040625
2,3.875090,0.008254,3.928343,0.028301
3,3.544505,0.020325,3.620146,0.031207
4,3.343664,0.021027,3.437796,0.027085
...,...,...,...,...
422,0.819005,0.013750,2.435361,0.026368
423,0.817867,0.013850,2.435400,0.026411
424,0.817031,0.013692,2.435299,0.026394
425,0.815601,0.013593,2.434916,0.026119


In [64]:
cv_results['test-mae-mean'].min()

2.4347438

In [65]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(7,12)
    for min_child_weight in range(5,8)
]
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=7, min_child_weight=5
[11:08:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:08:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:08:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:08:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:08:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 2.4012514000000005 for 309 rounds
CV with max_depth=7, min_child_weigh

In [66]:
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [69]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[11:15:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:15:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:15:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:15:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:15:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 2.2869468 for 153 rounds
CV with subsample=1.0, colsample=0.9
[11:16:03] 

In [70]:
params['subsample'] = best_params[0]
params['colsample_bytree'] = best_params[1]

In [71]:
tuned_model_jk = XGBRegressor(random_state=42, n_jobs=-1, max_depth = params['max_depth'], min_child_weight = params['min_child_weight'], subsample = params['subsample'], colsample_bytree =params['colsample_bytree'])

#Fit the algorithm on the data
tuned_model_jk.fit(training_merge_jk[:500000][jakarta_features], training_merge_jk[:500000][jakarta_target])

#Predict training set:
preds = tuned_model_jk.predict(training_merge_jk[:500000][jakarta_features])

#Evaluation
error = mean_squared_error(preds, training_merge_jk[:500000][jakarta_target], squared=False)

print("Training mean error for jk after tuning is {}".format(error))


#Validation
preds = tuned_model_jk.predict(validation_merge_jk[:500000][jakarta_features])

error_val = mean_squared_error(validation_merge_jk[:500000][jakarta_target], preds,squared = False)

print("Validation mean error for jk after tuning is {}".format(error_val))

C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Training mean error for jk is 2.7149837017059326


C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Validation mean error for jkis 3.877992868423462


> #### Now that we have reduced the mean error for training and validation datasets, we will be applying this tuned model to the entire Jakarta test dataset to observe whether there has been any improvement

In [72]:
tuned_model_jk = XGBRegressor(random_state=42, n_jobs=-1, max_depth = params['max_depth'], min_child_weight = params['min_child_weight'], subsample = params['subsample'], colsample_bytree =params['colsample_bytree'])

#Fit the algorithm on the data
tuned_model_jk.fit(test_merge_jk[jakarta_features], test_merge_jk[jakarta_target])

#Predict training set:
preds = tuned_model_jk.predict(test_merge_jk[jakarta_features])

#Evaluation
final_test_error_jk = mean_squared_error(preds, test_merge_jk[jakarta_target], squared=False)

print("Test mean error for jk is {}".format(final_test_error_jk))

C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\kimhy\anaconda3\envs\myenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Test mean error for jk is 3.868393898010254


> #### It seems that after hyperparameter tuning, there has been an observable improvement in our test error, from an initial value of 4.37 validation error to a final value of 3.86. 

### 4. Let's analyze the model and showcase how we can utilize the model to help alleviate Grab's problem

In [90]:
temp = list(zip(tuned_model_sg.get_booster().feature_names, tuned_model_sg.feature_importances_))
feature_importances_sg_df = pd.DataFrame(temp, columns =['Feature', 'Importance'])
feature_importances_sg_df.sort_values(by='Importance', ascending=False)

,Feature,Importance
1,rawlng,0.394675
0,rawlat,0.373960
2,time,0.145504
3,day_of_week,0.085861


In [91]:
temp = list(zip(tuned_model_jk.get_booster().feature_names, tuned_model_jk.feature_importances_))
feature_importances_jk_df = pd.DataFrame(temp, columns =['Feature', 'Importance'])
feature_importances_jk_df.sort_values(by='Importance', ascending=False)

,Feature,Importance
1,rawlng,0.279888
0,rawlat,0.278713
4,car,0.233970
2,time,0.116278
3,day_of_week,0.091151
5,motorcycle,0.000000


> #### As the above dataframes show, the latitude and longitude both have the highest importance when it comes to determining the speed of the driver, which is logically sound. For Singapore, the next important value is time which is too logically sound and for Jakarta, the mode of transport is deemed more important than time, which looks suspicious and deserves a thorough observation. 

In [130]:
hypo_data_sg = np.array([[1.28366053202], [103.850916596], [18], [4]])

In [134]:
prediction_sg = tuned_model_sg.predict(hypo_data_sg.reshape(1,-1))
prediction_sg[0]
print("The Singapore model forecasts that at Raffles City, on a typical Friday evening, 6pm, the speed would be roughly {:.2f}m/s or {:.2f}km/h".format(prediction_sg[0], prediction_sg[0]*(18/5)))

The Singapore model forecasts that at Raffles City, on a typical Friday evening, 6pm, the speed would be roughly 5.65m/s or 20.35km/h


> #### As we can see, inputting the coordinates of Raffles City in a typical rush hour, 6pm, on a Friday, predicted a slow speed of 5.65m/s (20.35km/h), which is way below the average value of speed and even lower than the 25 percentile of the speed distribution (11.6m/s). 

In [150]:
hypo_data_jk_car = np.array([[-6.245237], [106.783600], [18], [4], [1], [0]]) 
hypo_data_jk_motorcycle = np.array([[-6.245237], [106.783600], [18], [4], [0], [1]]) 

In [152]:
prediction_jk_car = tuned_model_jk.predict(hypo_data_jk_car.reshape(1,-1))
prediction_jk_motorcycle = tuned_model_jk.predict(hypo_data_jk_motorcycle.reshape(1,-1))

print("The Jakarta model forecasts that in Gandaria City, Kebayoran, the speed for car would be {:.2f}m/s or {:.2f}km/h and the speed for a motorcycle would be {:.2f}m/s or {:.2f}km/h".format(prediction_jk_car[0], prediction_jk_car[0]*(18/5), prediction_jk_motorcycle[0], prediction_jk_motorcycle[0]*(18/5)))

The Jakarta model forecasts that in Gandaria City, Kebayoran, the speed for car would be 6.10m/s or 21.97km/h and the speed for a motorcycle would be 6.12m/s or 22.04km/h


> #### As we can see, inputting the coordinates for a district known for its shopping malls on a Friday, 6pm, yields a slow speed of only about 20km/h, both for cars and motorcycles. We regret that there is a limitation here as our knowledge of Jakarta is not great enough to confidently point out where there would be greatest amount of congestion to try and test out our model. If we had more time, we would have loved to research more about local Jakarta and test out our model based on a specific location where we predict that there would be a lot of congestion

### 5. Limitations

### We have several limitations that we wish to acknowledge in terms of our model and our approach 

> #### In terms of our approach, the target variable felt good in theory as lower speed usually may relate to a congested road. However, there can be numerous other scenarios of stopping for traffic lights, turning corners or just coincidental drivers' choice to drive slow. If we could have come up with a method to mitigate these possible scenarios and engineer the 'speed' feature to be more accurate in showing congestion, we believe our model would have done much better 
> #### Additionally, apart from speed, maybe the density of drivers in a specific region, bounded by a range of latitude and longitude could have been a better gauge for congestion in certain parts of the region. 

> #### As for the model wise, yes it predicts better than a naive prediction model that simply predicts the mean value for speed. Furthermore, for the Singapore model, it shows a visible improvement in the error while for the Jakarta model, there is only a very slight improvement overall. However, the improvement isn't as drastic as we were expecting and we do not believe it to be of a high enough confidence level for Grab to potentially utilize it in improving ETA estimations and suggesting faster routes, which was our main goal initially. 

### 6. Future Improvements

> #### As for the modelwise, if time allowed, we believe we could have tried out some other supervised learning models such as SVM models, by normalizing feature variables and possibly using a non-linear kernel as we don't believe the distinctions would have been linear. For our current model of Gradient Boosting, we believe more relevant features would have presented us with a greater reduction in error and a more accurate model. 
> #### Additionally as mentioned above, we believe coming up with different gauges of congestion on the road and using them as target variables for different models could have potentially led to different and maybe even improved results. 
> #### Also for our variable time, which is expressing the hour of the day, I specifying to the minute values would have possibly painted a more accurate picture. 